In [1]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import matplotlib.pyplot as plt

# Load & Explore the dataset

The **pbmc3k dataset** is a widely used single-cell RNA-seq dataset that contains around 3,000 peripheral blood mononuclear cells (PBMCs) from a healthy donor. This dataset was generated by 10X Genomics and is commonly used as a benchmark in single-cell analysis pipelines. PBMCs are a diverse collection of immune cells including T cells, B cells, natural killer (NK) cells, and monocytes.

## Dataset Construction

The pbmc3k dataset is provided in the form of an **AnnData** object, a popular data structure in the Scanpy ecosystem designed specifically for single-cell data. Key components of the AnnData object include:

- **adata.X**: The main data matrix where each row represents a cell and each column represents a gene.
- **adata.obs**: A pandas DataFrame that contains metadata for each cell (e.g., total counts, number of detected genes).
- **adata.var**: A pandas DataFrame that holds metadata for each gene (e.g., gene names, whether a gene is mitochondrial).

Below, we load and explore the pbmc3k dataset.


In [10]:
# Load the pbmc3k dataset
adata = sc.datasets.pbmc3k()

# print the basic information of the dataset
print("Basic information:")
print(adata)

# View the first few rows of the cell metadata (obs)
print("\nCell Metadata (obs): ")
print(adata.obs.head())

# View the first few rows of the gene metadata (var)
print("\nGene Metadata (var):")
print(adata.var.head())

# Print the shape of the main data matrix (cells x genes)
print("\nData Matrix Shape:")
print(adata.X.shape)

Basic information:
AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

Cell Metadata (obs): 
Empty DataFrame
Columns: []
Index: [AAACATACAACCAC-1, AAACATTGAGCTAC-1, AAACATTGATCAGC-1, AAACCGTGCTTCCG-1, AAACCGTGTATGCG-1]

Gene Metadata (var):
                     gene_ids
index                        
MIR1302-10    ENSG00000243485
FAM138A       ENSG00000237613
OR4F5         ENSG00000186092
RP11-34P13.7  ENSG00000238009
RP11-34P13.8  ENSG00000239945

Data Matrix Shape:
(2700, 32738)


## pbmc3k Dataset Description

Here are its key characteristics:

- **Number of Cells (`n_obs`):** 2,700  
- **Number of Genes (`n_vars`):** 32,738  
- **Data Structure:** Stored in an AnnData object, which is a common format for single-cell data in Python.

AnnData Structure:

1. **`adata.obs` (Cell Metadata):**  
   - **Index:** Unique cell barcodes (e.g., `AAACATACAACCAC-1`).  
   - **Columns:** Empty in this particular dataset, as no additional cell-level annotations are provided by default.

2. **`adata.var` (Gene Metadata):**  
   - **Index:** Gene symbols or IDs (e.g., `MIR1302-10`).  
   - **Columns:** Typically contains at least one column, here named `"gene_ids"`, which stores the corresponding Ensembl gene IDs.

3. **`adata.X` (Main Data Matrix):**  
   - A matrix of size `n_obs x n_vars` (2,700 × 32,738 in this case).  
   - Each row corresponds to a cell, and each column corresponds to a gene.  
   - The values are gene expression counts (or sometimes transformed counts).
      - **Gene Expression Counts**: Raw counts (also called UMI counts in many single-cell protocols) represent the number of times a given transcript (gene) was detected in each cell.
      - **Transformed Counts**: Transformed counts are derived from raw counts through operations such as normalization and log-transformation. 

# Simulate Batch Effects
We simulate batch effects by dividing the dataset into two artificial batches.

## Why do we simulate Batch Effects?
- **Testing Batch Correction Methods:**
The pbmc3k dataset originally comes from a single batch, meaning it lacks inherent batch effects. By artificially dividing the data, we create a scenario where batch effects are present. This allows us to test and validate batch integration methods (like Scanorama) in a controlled setting.

- **Mimicking Real-World Data:**
Many real-world single-cell experiments involve multiple batches. Simulating batches helps us understand how our analysis pipeline would perform on data where batch-to-batch variability is a concern.

## Why Only Two Batches? 
   - **Common Scenario:** Many single-cell experiments involve two batches, especially when comparing different experimental conditions or replicates.
   - **Simplicity in Demonstration:** Using two batches keeps the example straightforward while still demonstrating the need for batch integration. More complex scenarios can be explored by creating additional artificial batches if needed.

In [ ]:
# The number of cells
n_cells = adata.n_obs

# Split into two equal halves based on indices
adata.obs['batch'] = np.where(np.arange(n_cells) < n_cells / 2, 'batch1', 'batch2')
print(adata.obs)

                   batch
index                   
AAACATACAACCAC-1  batch1
AAACATTGAGCTAC-1  batch1
AAACATTGATCAGC-1  batch1
AAACCGTGCTTCCG-1  batch1
AAACCGTGTATGCG-1  batch1
...                  ...
TTTCGAACTCTCAT-1  batch2
TTTCTACTGAGGCA-1  batch2
TTTCTACTTCCTCG-1  batch2
TTTGCATGAGAGGC-1  batch2
TTTGCATGCCTCAC-1  batch2

[2700 rows x 1 columns]


In this example, we are using a simple, straightforward criterion to split the dataset into two artificial batches:
   - `np.arange(n_cells)` generates an array of indices from 0 to `n_cells - 1`.
   - The condition `< n_cells / 2` splits this array into two equal halves.
   - Cells with indices in the first half are labeled as `'batch1'`, and the rest are labeled as `'batch2'`.

## Why This Specific Criterion?
   - **Simplicity:** Dividing the dataset by the index is an easy and reproducible way to split the data evenly.
   - **Balance:** By splitting the cells equally, we ensure that each batch has a similar number of cells, which is useful for comparing the performance of batch correction methods.
   - **Controlled Simulation:** Since the original pbmc3k dataset does not have natural batch effects, this method allows us to simulate a batch effect scenario in a controlled manner.


# Quality Control 

In this step, we compute QC metrics including:
- Total counts per cell (The sum of all UMIs)
- Number of genes detected per cell
- Percentage of mitochondrial gene counts

We visualize these metrics using plots and filter out considering MAD (median absolute deviations).


In [35]:
# Mark mitochondrial genes (gene names starting with 'MT-' since the dataset is about human cells)
adata.var['mt'] = adata.var_names.str.startswith('MT-')

# Compute QC metrics for each cell
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)

print(adata.obs.head())

                   batch  n_genes_by_counts  log1p_n_genes_by_counts  \
index                                                                  
AAACATACAACCAC-1  batch1                781                 6.661855   
AAACATTGAGCTAC-1  batch1               1352                 7.210080   
AAACATTGATCAGC-1  batch1               1131                 7.031741   
AAACCGTGCTTCCG-1  batch1                960                 6.867974   
AAACCGTGTATGCG-1  batch1                522                 6.259581   

                  total_counts  log1p_total_counts  \
index                                                
AAACATACAACCAC-1        2421.0            7.792349   
AAACATTGAGCTAC-1        4903.0            8.497807   
AAACATTGATCAGC-1        3149.0            8.055158   
AAACCGTGCTTCCG-1        2639.0            7.878534   
AAACCGTGTATGCG-1         981.0            6.889591   

                  pct_counts_in_top_50_genes  pct_counts_in_top_100_genes  \
index                                  

                     gene_ids     mt  n_cells_by_counts  mean_counts  \
index                                                                  
MIR1302-10    ENSG00000243485  False                  0          0.0   
FAM138A       ENSG00000237613  False                  0          0.0   
OR4F5         ENSG00000186092  False                  0          0.0   
RP11-34P13.7  ENSG00000238009  False                  0          0.0   
RP11-34P13.8  ENSG00000239945  False                  0          0.0   
...                       ...    ...                ...          ...   
AC145205.1    ENSG00000215635  False                  0          0.0   
BAGE5         ENSG00000268590  False                  0          0.0   
CU459201.1    ENSG00000251180  False                  0          0.0   
AC002321.2    ENSG00000215616  False                  0          0.0   
AC002321.1    ENSG00000215611  False                  0          0.0   

              log1p_mean_counts  pct_dropout_by_counts  total_c